In [84]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.isotonic import IsotonicRegression

In [85]:
## Loading the dataset
data = pd.read_csv("../Data/DB/Month_of_July.csv")
data = data.drop("Unnamed: 0",axis=1)

In [86]:
data[["brand"]].value_counts()

brand     
TOYOTA        2449
SUZUKI         860
NISSAN         819
HONDA          606
MITSUBISHI     586
BMW            399
DAIHATSU       218
KIA            206
HYUNDAI        201
MAHINDRA       188
MICRO          140
MAZDA          139
PERODUA         88
TATA            73
BYD             20
VOLKSWAGEN       1
dtype: int64

## Brand



In [87]:
data

,brand,model,year,Transmission,fuel_type,Engine capacity,Mileage,vehicle_price
0,HONDA,VEZEL,2014,AUTOMATIC,HYBRID,1500,102000,10400000
1,HONDA,VEZEL,2023,AUTOMATIC,HYBRID,1500,7000,23800000
2,HONDA,GRACE,2015,AUTOMATIC,HYBRID,1500,185000,10500000
3,HONDA,VEZEL,2017,AUTOMATIC,HYBRID,1500,112000,13600000
4,HONDA,VEZEL,2018,AUTOMATIC,HYBRID,1500,53600,15000000
...,...,...,...,...,...,...,...,...
6988,BYD,SEAL DYNAMIC,2025,AUTOMATIC,ELECTRIC,1000,0,19700000
6989,BYD,SEAL PREMIUM,2025,AUTOMATIC,ELECTRIC,1000,0,54700000
6990,BYD,SEAL PERFORMANCE,2025,AUTOMATIC,ELECTRIC,1000,0,78050000
6991,BYD,SHARK PREMIUM,2025,AUTOMATIC,ELECTRIC,1000,0,25300000


In [78]:
## Filter the brand name
brand_name = "NISSAN"
brand = data[data['brand']=='NISSAN']
brand = brand[brand['vehicle_price'] >= 1000000]


In [80]:
brand

,brand,model,year,Transmission,fuel_type,Engine capacity,Mileage,vehicle_price
1563,NISSAN,DAYZ,2019,AUTOMATIC,PETROL,660,24800,8100000
1564,NISSAN,MARCH,1999,AUTOMATIC,PETROL,999,146000,3390000
1565,NISSAN,QASHQAI,2018,AUTOMATIC,PETROL,1200,150000,14000000
1566,NISSAN,MARCH,2011,AUTOMATIC,PETROL,1240,115000,4850000
1567,NISSAN,MARCH,2000,AUTOMATIC,PETROL,1000,120000,2875000
...,...,...,...,...,...,...,...,...
6060,NISSAN,DUTSUN,1982,MANUAL,DIESEL,2164,361200,1000000
6062,NISSAN,WINGROAD,1998,MANUAL,DIESEL,1970,213000,3000000
6063,NISSAN,DOUBLE CAB,2006,MANUAL,DIESEL,2700,295000,2500000
6064,NISSAN,QASHQAI,2018,TIPTRONIC,PETROL,1200,85000,14500000


In [81]:
brand_model_year = brand[["brand","model","year"]]

# Get unique combinations
unique_combinations_brand = brand_model_year.drop_duplicates().reset_index(drop=True)
# Assuming your DataFrame is named df
grouped = unique_combinations_brand.groupby(['brand', 'model'])['year'].count().reset_index()
grouped_df = grouped.rename(columns={'year': 'count'})

# Filter models with count >= 5
filtered = grouped_df[grouped_df['model'] != 'OTHER MODEL']
filtered = filtered[filtered['count'] >= 5].reset_index(drop=True)

# Get the unique model names into a list
brand_model_list = filtered['model'].unique().tolist()


In [82]:
brand_model_list

['AD WAGON',
 'BLUEBIRD',
 'CARAVAN',
 'CEFIRO',
 'D21',
 'DAYZ',
 'DOUBLE CAB',
 'DUTSUN',
 'FB14',
 'LEAF',
 'MARCH',
 'MARCH K11',
 'NAVARA',
 'PATROL',
 'PULSAR',
 'QASHQAI',
 'SUNNY',
 'TRAD SUNNY',
 'VANETTE',
 'WINGROAD',
 'X-TRAIL',
 'XTRAIL']

In [68]:
## get the unique models 
top_brand_models = unique_combinations_brand[unique_combinations_brand['model'].isin(brand_model_list)]

## get the median of vehicle prices

brand_df = brand.groupby(["brand", "model", "year"]).median().reset_index()

year_range_df = brand_df.groupby(['brand', 'model'])['year'].agg(['min', 'max']).reset_index()

# Expand each row into a range of years
expanded_rows = []

for _, row in year_range_df.iterrows():
    for year in range(row['min'], row['max'] + 1):
        expanded_rows.append({
            'brand': row['brand'],
            'model': row['model'],
            'year': year
        })

# Create the expanded DataFrame
expanded_df = pd.DataFrame(expanded_rows)

C:\Users\pamith.rathnayake\AppData\Local\Temp\ipykernel_4128\1807233102.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  brand_df = brand.groupby(["brand", "model", "year"]).median().reset_index()


In [69]:
## joining the brand_df and expanded_df

final_brand = expanded_df.merge(brand_df, on=["brand", "model", "year"], how="left")

In [ ]:
df = final_brand.copy()


def fill_and_enforce_monotonic(group):
    group = group.sort_values('year').reset_index(drop=True)
    prices = group['vehicle_price'].values.copy()
    years = group['year'].values

    known_idx = np.where(~np.isnan(prices))[0]
    missing_idx = np.where(np.isnan(prices))[0]

    for i in missing_idx:
        lower_idx = known_idx[known_idx < i]
        upper_idx = known_idx[known_idx > i]
        if len(lower_idx) > 0 and len(upper_idx) > 0:
            low_i = lower_idx[-1]
            up_i = upper_idx[0]
            ratio = (years[i] - years[low_i]) / (years[up_i] - years[low_i])
            prices[i] = prices[low_i] + ratio * (prices[up_i] - prices[low_i])
        elif len(lower_idx) > 0:
            prices[i] = prices[lower_idx[-1]]
        elif len(upper_idx) > 0:
            prices[i] = prices[upper_idx[0]]
        else:
            prices[i] = np.nan  
    for i in range(1, len(prices)):
        if i in missing_idx:
           
            if prices[i] <= prices[i-1]:
                prices[i] = prices[i-1] + 1  

    group['vehicle_price'] = prices.astype(int)
    return group

df_filled = final_brand.groupby(['brand', 'model'], group_keys=False).apply(fill_and_enforce_monotonic)

print(df_filled)

     brand         model  year  vehicle_price
0   NISSAN          370Z  2013       35000000
0   NISSAN           4WD  1985        3685000
0   NISSAN           720  1985        1750000
0   NISSAN       AD VY11  2003        3385000
0   NISSAN      AD WAGON  1989        1212500
..     ...           ...   ...            ...
23  NISSAN        XTRAIL  2023       29000000
24  NISSAN        XTRAIL  2024       31500000
25  NISSAN        XTRAIL  2025       34000000
0   NISSAN  Y10 WINGROAD  1998        3500000
0   NISSAN           Y11  2001        3050000

[711 rows x 4 columns]


In [ ]:
df = df_filled.sort_values(['brand', 'model', 'year']).reset_index(drop=True)

def adjust_price(group):
    group = group.sort_values('year').reset_index(drop=True)
    adjusted = group['vehicle_price'].copy()

    for i in range(1, len(group)):
        prev_price = adjusted[i-1]
        curr_price = adjusted[i]
        if prev_price > curr_price:
            adjusted[i] = prev_price * 1.03  

    group['adjusted_price'] = adjusted.round().astype(int)
    return group

df = df.groupby(['brand', 'model'], group_keys=False).apply(adjust_price)

print(df[['brand', 'model', 'year', 'vehicle_price', 'adjusted_price']])

     brand         model  year  vehicle_price  adjusted_price
0   NISSAN          370Z  2013       35000000        35000000
0   NISSAN           4WD  1985        3685000         3685000
0   NISSAN           720  1985        1750000         1750000
0   NISSAN       AD VY11  2003        3385000         3385000
0   NISSAN      AD WAGON  1989        1212500         1212500
..     ...           ...   ...            ...             ...
23  NISSAN        XTRAIL  2023       29000000        29000000
24  NISSAN        XTRAIL  2024       31500000        31500000
25  NISSAN        XTRAIL  2025       34000000        34000000
0   NISSAN  Y10 WINGROAD  1998        3500000         3500000
0   NISSAN           Y11  2001        3050000         3050000

[711 rows x 5 columns]


In [72]:
df

,brand,model,year,vehicle_price,adjusted_price
0,NISSAN,370Z,2013,35000000,35000000
0,NISSAN,4WD,1985,3685000,3685000
0,NISSAN,720,1985,1750000,1750000
0,NISSAN,AD VY11,2003,3385000,3385000
0,NISSAN,AD WAGON,1989,1212500,1212500
...,...,...,...,...,...
23,NISSAN,XTRAIL,2023,29000000,29000000
24,NISSAN,XTRAIL,2024,31500000,31500000
25,NISSAN,XTRAIL,2025,34000000,34000000
0,NISSAN,Y10 WINGROAD,1998,3500000,3500000


In [58]:
df.to_csv("Nissan.csv")